# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files 

**Date**: October 10th 2025

**Student Name**: 

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Laboratorio7 Pablo Quintero") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")
spark.conf.set("spark.sql.shuffle.partitions", "5")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/11 03:41:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
from pyspark.sql.functions import split, col, window, expr
from pyspark.sql.types import StructType, StringType, IntegerType, TimestampType
from pyspark.sql.functions import regexp_replace, to_timestamp

schema = StructType() \
    .add("ip", StringType()) \
    .add("dash1", StringType()) \
    .add("dash2", StringType()) \
    .add("timestamp_str", StringType()) \
    .add("request", StringType()) \
    .add("status", IntegerType()) \
    .add("size", IntegerType())


lines = spark.readStream.format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()
logs = lines.selectExpr("value as raw") \
    .select(
        split(col("raw"), " ")[0].alias("ip"),
        split(col("raw"), " ")[1].alias("dash1"),
        split(col("raw"), " ")[2].alias("dash2"),
        split(col("raw"), " ")[3].alias("timestamp_str"),
        split(col("raw"), "\"")[1].alias("request"),
        split(col("raw"), " ")[-2].cast("integer").alias("status"),
        split(col("raw"), " ")[-1].cast("integer").alias("size")
    )

logs = logs.withColumn("timestamp_str", regexp_replace("timestamp_str", r"[\[\]]", ""))
logs = logs.withColumn("timestamp", to_timestamp("timestamp_str", "yyyy-MM-dd HH:mm:ss"))


error500 = logs.filter(col("status") == 500)

errors_window = error500.groupBy(
    window(col("timestamp"), "1 minute"),
    col("ip")
).count()


critical_alerts = errors_window.filter(col("count") >= 3)


query = critical_alerts.writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", False) \
    .start()


query.awaitTermination(30) 

In [ ]:
!python3 log_producer.py

